In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from client import Client

In [28]:
#NODEs
client1 = Client("node1","0.0.0.0")
dataset = client1.load_data("../datasets/UNSW_NB15_training-set.csv")
client2 = Client("node2","0.0.0.0")
client3 = Client("node3","0.0.0.0")

clients = [client1, client2, client3]

client1.preprocess_data(dataset)

X = client1.x
y = client1.y

client1.x = X[:20000]
client1.y = y[:20000]

client2.x = X[20000:40000]
client2.y = y[20000:40000]

client3.x = X[40000:60000]
client3.y = y[40000:60000]

x_test = X[60000:]
y_test = y[60000:]

In [30]:
for client in clients:
    client.train_model()
    print(f'Model accuracy: {client.accuracy}')
    print(f'Model f1 score: {client.F1}')

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model accuracy: 0.9884848484848485
Model f1 score: 0.994206433907608


/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tk/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

Model accuracy: 0.9978787878787879
Model f1 score: 0.9935364727608494
Model accuracy: 0.826060606060606
Model f1 score: 0.9038364885240409


In [31]:
number_of_rounds = 5
for _ in range(number_of_rounds):
    print(f'Starting new round!')
    f1_local1 = client1.test_model_f1(y_test,x_test); print(f1_local1)
    f1_local2 = client2.test_model_f1(y_test,x_test); print(f1_local2)
    f1_local3 = client3.test_model_f1(y_test,x_test); print(f1_local3)
    
    w1 = f1_local1/(f1_local1+f1_local2+f1_local3); print(w1)
    w2 = f1_local2/(f1_local1+f1_local2+f1_local3); print(w2)
    w3 = f1_local3/(f1_local1+f1_local2+f1_local3); print(w3)
    
    intercept_fl = w1*client1.model.intercept_ + w2*client2.model.intercept_ + w3*client2.model.intercept_
    coefs_fl = w1*client1.model.coef_ + w2*client2.model.coef_ + + w3*client2.model.coef_
    
    for client in clients:
        f1_local = client.test_model_accuracy(y_test,x_test)
        int_local = client.model.intercept_
        coef_local = client.model.coef_
        print("f1 local: ", f1_local)

        client1.model.intercept_ = intercept_fl
        client.model.coef_ = coefs_fl
        f1_fede = client.test_model_accuracy(y_test,x_test)
        print("f1 fede: ", f1_fede)
        if f1_fede < f1_local:
            client.model.intercept_ = int_local
            client.model.coef_ = coef_local
            print("f1 local was better")
            print(f1_local)

        elif f1_fede == f1_local:
            print("the same")

        else:
            print("f1 fl was better")
            print(f1_fede)
    print(f'++++++++++++++++++++++++++++++++++++')

Starting new round!
0.41513183576422164
0.5115710418304079
0.41461505621175465
0.30949547851504744
0.3813943204193763
0.3091102010655762
acc local:  0.26200071646068424
acc fede:  0.7492835393157801
f1 fl was better
0.7492835393157801
acc local:  0.7495522120723626
acc fede:  0.7492835393157801
f1 local was better
0.7495522120723626
acc local:  0.2632097438653054
acc fede:  0.7492835393157801
f1 fl was better
0.7492835393157801
++++++++++++++++++++++++++++++++++++
Starting new round!
0.5160342294061717
0.5115710418304079
0.5160342294061717
0.33429711353674335
0.3314057729265132
0.33429711353674335
acc local:  0.7492835393157801
acc fede:  0.74964176965789
f1 fl was better
0.74964176965789
acc local:  0.7495522120723626
acc fede:  0.74964176965789
f1 fl was better
0.74964176965789
acc local:  0.7492835393157801
acc fede:  0.74964176965789
f1 fl was better
0.74964176965789
++++++++++++++++++++++++++++++++++++
Starting new round!
0.5135299747672496
0.5135299747672496
0.5135299747672496
0.

In [13]:
def init_global_model():
    fedavg = Client("global","1.1.1.1")
    fedavg.model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                       l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                       random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
    fedavg.model.intercept_ = np.zeros(1)
    fedavg.model.coef_ = np.zeros((1,X.shape[1]))
    fedavg.classes_ = np.array([0,1])

In [22]:
client1.accuracy
client2.accuracy

0.7857575757575758

In [23]:

rounds = 50
epochs = 10
batch_size = 100

In [24]:
sample_size = 2
remove_agents = []

In [ ]:
weights = []

for client in clients:
    n_samples = client.x.shape[0]
    scan = sum(client.y["label"] == 1)
    w = n_samples / (2*scan)
    weights.append(w)

In [26]:
n_iterations = 50

In [28]:
import random


for i in range(0,n_iterations):
    init_global_model()

    for r in range(rounds):
        applicable_agents = random.sample(clients,k=2)

        applicable_models = []
        applicable_name = []

        #weight average
        round_weights = []
        dataset_size = 0

        for client in 

NameError: name 'init_global_model' is not defined